In [1]:
import cv2
import numpy as np
from pathlib import Path
from boxmot import DeepOCSORT
from ultralytics import YOLO

In [2]:
def initialize_tracker(model_weights, device='cuda:0', fp16=True):
    """
    Initialize the DeepOCSORT tracker with the given model weights and device.
    """
    tracker = DeepOCSORT(
        model_weights=model_weights,
        device=device,
        fp16=fp16
    )
    return tracker

def initialize_yolo(model_path):
    """
    Initialize the YOLO object detector with the given model path.
    """
    yolov8 = YOLO(model_path)
    return yolov8

In [3]:
class Rect:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
    
    @property
    def bottom_center(self):
        return ((self.x1 + self.x2) // 2, self.y2)
    
    @property
    def width(self):
        return self.x2 - self.x1

class Color:
    def __init__(self, b, g, r):
        self.b = b
        self.g = g
        self.r = r

    @property
    def bgr_tuple(self):
        return (self.b, self.g, self.r)

def draw_ellipse(image, rect, color, thickness=2):
    center = rect.bottom_center
    axes = (int(rect.width), int(0.35 * rect.width))
    cv2.ellipse(
        image,
        center,
        axes,
        angle=0.0,
        startAngle=-45,
        endAngle=235,
        color=color.bgr_tuple,
        thickness=thickness,
        lineType=cv2.LINE_4
    )
    return image

def lab_to_rgb(lab_color):
    """
    Converts a single LAB color to RGB.
    """
    lab_image = np.array([[lab_color]], dtype=np.uint8)
    rgb_image = cv2.cvtColor(lab_image, cv2.COLOR_Lab2RGB)
    return rgb_image[0][0]

In [4]:
def process_video(input_file, output_file, tracker, yolov8, conf=0.3):
    """
    Process the input video, perform object detection and tracking, and save the results to the output video.
    """
    input_video = cv2.VideoCapture(input_file)
    width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = input_video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
    
    thickness = 2
    fontscale = 0.5
    
    # Function to get unique colors for each class id
    def get_color(cls_id):
        np.random.seed(cls_id)
        return tuple(np.random.randint(0, 255, 3).tolist())
    
   

    initial_centroids = None
    counter = 1
    while True:
        print(counter)
        ret, im = input_video.read()
        if not ret:
            break
        
        predicted = yolov8.predict(im, conf=conf)
        dets = predicted[0].boxes.data.cpu().numpy()
        """
        tracks = tracker.update(dets, im)
        if tracks.size > 0:  # Check if tracks is not empty
            print("Tracks shape:", tracks.shape)  # Print the shape of tracks
            xyxys = tracks[:, 0:4].astype('int')
            ids = tracks[:, 4].astype('int')
            clss = tracks[:, 6].astype('int')
        else:
            print("Tracks is empty or not in expected format.")
        
        #if(counter == 1):
        #   compute_jersey_colors(im, dets, xyxys, clss)
        
        if tracks.shape[0] != 0:
            #player_mean_colors = [get_mean_color(im, xyxy)[1] for xyxy, cls in zip(xyxys, clss) if cls == 1]
            #player_mean_colors = [get_color_histogram(im, xyxy) for xyxy, cls in zip(xyxys, clss) if cls == 1]
            
            #if len(player_mean_colors) > 1:
            #    labels, initial_centroids = perform_kmeans_clustering(player_mean_colors, initial_centroids)
                
                
                player_mean_colors = np.array([lab_to_rgb(color) for color in player_mean_colors])
                
                # Plotting the results (optional)
                plt.subplot(1, 2, 1)
                plt.scatter(player_mean_colors[:, 1], player_mean_colors[:, 2], c=player_mean_colors / 255.0)
                plt.title('Original Colors')

                plt.subplot(1, 2, 2)
                plt.scatter(player_mean_colors[labels == 0][:, 1], player_mean_colors[labels == 0][:, 2], c='blue', marker='o')
                plt.scatter(player_mean_colors[labels == 1][:, 1], player_mean_colors[labels == 1][:, 2], c='red', marker='x')
                plt.title('Clustered Colors')

                plt.show()
                
            """
            for xyxy, id, cls, label in zip(xyxys, ids, clss, labels):
                
                if cls == 1:  # Modify this condition as per your actual player class ID
                    # Adjusting the bounding box to focus on the upper part
                    x1, y1, x2, y2 = xyxy
                    height = y2 - y1
                    width = x2 - x1

                    new_y1 = y1 + int(0.17 * height)
                    new_y2 = y1 + int(0.5 * height)
                    new_x1 = x1 + int(0.2 * width)
                    new_x2 = x2 - int(0.2 * width)

                    # Visualize the adjusted bounding box
                    cv2.rectangle(im, (new_x1, new_y1), (new_x2, new_y2), (255, 0, 0), 2)
                
                
                # Convert bounding box coordinates to Rect object
                rect = Rect(xyxy[0], xyxy[1], xyxy[2], xyxy[3])
                
                if cls == 1:
                    cls = cls + label
                
                # Get a unique color for each class
                color = Color(*get_color(cls))

                # Draw an ellipse at the bottom of the bounding box
                draw_ellipse(im, rect, color, thickness)
                
                # Place ID text just above the ellipse
                cv2.putText(
                    im,
                    f'{id}',
                    (rect.bottom_center[0] - 10, rect.bottom_center[1] - int(0.35 * rect.width) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    fontscale,
                    color.bgr_tuple,
                    thickness
                )
                
                
        out.write(im)
        counter += 1
        
    input_video.release()
    out.release()
    cv2.destroyAllWindows()

In [5]:
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
model_weights = Path('osnet_x0_25_msmt17.pt')
yolo_model_path = 'runs/detect/NewDataset5008/weights/best.pt'
input_video_file = "videos/2sec.mp4"
print(input_video_file.split(".")[0] + "_tracking1_output.mp4")
output_video_file = input_video_file.split(".")[0] + "_tracking1_output.mp4"

tracker = initialize_tracker(model_weights)
yolov8 = initialize_yolo(yolo_model_path)
process_video(input_video_file, output_video_file, tracker, yolov8)

videos/2sec_tracking1_output.mp4


2023-10-16 12:53:22.145 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-10-16 12:53:22.146 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


1



0: 1120x1984 1 ball, 22 players, 77.6ms
Speed: 316.4ms preprocess, 77.6ms inference, 382.0ms postprocess per image at shape (1, 3, 1120, 1984)


Tracks shape: (23, 8)


NameError: name 'compute_jersey_colors' is not defined